# Sistema de Recomendación de Artistas

![Music4U](Music4U_logo.png)

**Importamos las librerías necesarias**

In [1]:
#Librerias necesarias
import pandas as pd
import zipfile as zp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import widgets, Button, interact_manual
import plotly.offline as py 
from plotly.figure_factory import create_table
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from IPython.display import display, clear_output

**Importamos la tabla de canciones y la de artistas con géneros, ya que serán las dos tablas de la [base de datos de Spotify](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks) que usaremos para este sistema de recomendación**

In [2]:
#Importar las tablas de la base de datos
archive = zp.ZipFile("archive.zip", "r")
original_df_genero = pd.read_csv(archive.open("data_w_genres.csv"))
original_df_genero.head(1)

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,"""Cats"" 1981 Original London Cast",0.575083,0.44275,247260.0,0.386336,0.022717,0.287708,-14.205417,0.180675,115.9835,0.334433,38.0,5,1,12,['show tunes']


**Preparamos los datos en la tabla de artistas con géneros**

In [3]:
#Eliminar los generos nulos
no_generos_nulos = original_df_genero[original_df_genero.genres != "[]"]
#Copiar la nueva tabla sin generos nulos a una nueva tabla con un nombre nuevo
df_genero = no_generos_nulos.copy()
#Convertir la popularidad de float a entero
df_genero['popularity'] = df_genero['popularity'].astype(int)
df_genero = df_genero.reset_index(drop=True)
#Convertir las listas de string en la columna de generos en listas de verdad
df_genero['genres'] = df_genero['genres'].apply(lambda x: x[1:-1].split(', '))
df_genero.rename(columns={'popularity':'Popularidad Artista'}, inplace=True)
df_genero.head(1)

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,Popularidad Artista,key,mode,count,genres
0,"""Cats"" 1981 Original London Cast",0.575083,0.44275,247260.0,0.386336,0.022717,0.287708,-14.205417,0.180675,115.9835,0.334433,38,5,1,12,['show tunes']


In [4]:
#Eliminamos las columnas que no necesitamos
new_df_genero = df_genero.drop(['duration_ms', 'count'], axis=1)
#Reordenamos las columnas de la siguiente manera para poder visualizar mejor los datos
columns_reorder = ['artists', 'Popularidad Artista', 'genres', 'acousticness', 'danceability', 'energy', 
                   'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence']

new_df_genero = new_df_genero.reindex(columns=columns_reorder)

new_df_genero.head(1)

,artists,Popularidad Artista,genres,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,"""Cats"" 1981 Original London Cast",38,['show tunes'],0.575083,0.44275,0.386336,0.022717,5,0.287708,-14.205417,1,0.180675,115.9835,0.334433


In [5]:
#Separamos las columnas musicales numericas en una nueva tabla
song_data = new_df_genero.loc[:, ['acousticness', 'danceability', 'energy', 'instrumentalness', 
                              'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                             'valence']]

#MinMaxScaler: Transforma las características escalando cada característica a un rango determinado.
#Este estimador escala y traduce cada característica individualmente de manera que se encuentre en el 
#rango dado en el conjunto de entrenamiento, en nuestro caso de -1 a 1.
scaler = MinMaxScaler()

song_features = pd.DataFrame()

for col in song_data.iloc[:, :].columns:
    scaler.fit(song_data[[col]])
    song_features[col] = scaler.transform(song_data[col].values.reshape(-1,1)).ravel()

song_features.head(1)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.577393,0.449037,0.386336,0.022717,0.454545,0.290321,0.720962,1.0,0.188596,0.532662,0.337471


In [6]:
#Ahora que nuestros datos ya están escalados, eliminamos las columnas musicales numéricas en nuestra tabla new_df_genero
#y las reemplazamos por las nuevas escaladas
reemplazar_datos = new_df_genero.drop(['acousticness', 'danceability', 'energy', 'instrumentalness', 
                              'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                             'valence'], axis=1)
                               
final_df_genre = reemplazar_datos.join(song_features)
final_df_genre.head(1)

,artists,Popularidad Artista,genres,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,"""Cats"" 1981 Original London Cast",38,['show tunes'],0.577393,0.449037,0.386336,0.022717,0.454545,0.290321,0.720962,1.0,0.188596,0.532662,0.337471


**Ya que los datos estan preparados procedemos a realizar nuestro sistema de recomendacion**

In [7]:
#Funcion que realizara el sistema de recomendacion de artistas
def sistema_recomendacion_artistas(boton):
    with outuser_info:
        clear_output()

        #Verificamos que el artista ingresado exista en la columna de artistas de la tabla de artistas con generos
        datos_artista = final_df_genre[final_df_genre.artists == artista_txtbox.value]
        #Obtenemos los generos que contiene ese artista (es una lista)
        generos_artista = set(*datos_artista.genres)

        #Hacemos una copia de la tabla original para poder trabajar sobre ella
        similitud_datos = final_df_genre.copy()

        #Obtenemos solamente los datos musicales numericos, pues con estos trabajaremos para calcular la similitud del coseno
        valores_datos = similitud_datos.loc[:, ['acousticness', 'danceability', 'energy', 'instrumentalness', 
                                  'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                                 'valence']]

        #Creamos una nueva columna que se llame 'Similitud con Artista'
        #Posteriormente aqui realizaremos la similitud del coseno con los datos musicales numericos que extrajimos
        #La similitud del coseno se aplica a todos los artistas
        similitud_datos['Similitud con Artista'] = cosine_similarity(valores_datos, valores_datos.to_numpy()[datos_artista.index[0], None]).squeeze()


        #Buscamos una interseccion entre todos los generos de la tabla con los generos de nuestro artista ingresado 
        similitud_datos.genres = similitud_datos.genres.apply(lambda genres: list(set(genres).intersection(generos_artista)))


        similitud_longitud = similitud_datos.genres.str.len()
        #Dado el parametro del genero esta variable obtenra mas generos o el mismo numero de generos que deseemos 
        similitud_datos = similitud_datos.reindex(similitud_longitud[similitud_longitud >= genero_slider.value].sort_values(ascending=False).index)

        similitud_datos = similitud_datos.sort_values(by = 'Similitud con Artista', ascending=False)

        #Renombramos la columnas de artistas y generos
        similitud_datos.rename(columns={'artists':'Artista Similar', 'genres':'Generos'}, inplace=True)

        
        #Creamos una tabla que solo contenga los primeros 10 elementos para trabajar con ella
        tabla_grafica = similitud_datos.head(10)
        
        #Creamos una grafica con estos nuevos datos que acabamos de obtener
        size = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10]
        fig = go.Figure(data=[go.Scatter(
            x=tabla_grafica['Similitud con Artista'], y=tabla_grafica['Artista Similar'], 
            mode='markers',
            marker=dict(
                color=['rgb(255, 241, 0)', 'rgb(255, 140, 0)', 'rgb(232, 17, 35)', 'rgb(236, 0, 140)',
                      'rgb(104, 33, 122)', 'rgb(0, 24, 143)', 'rgb(0, 188, 242)', 'rgb(0, 178, 148)',
                      'rgb(0, 158, 73)', 'rgb(186, 216, 10)'],
                size=size,
            ),
        )])
        fig.update_layout(title='Artistas similares a ' + artista_txtbox.value)
        
        #Mostramos una tabla y una grafica con el sistema de recomendacion
        #return similitud_datos.loc[:, ['Artista Similar', 'Popularidad Artista', 'Generos', 'Similitud con Artista']].head(10)
        display(similitud_datos.loc[:, ['Artista Similar', 'Popularidad Artista', 'Generos', 'Similitud con Artista']].head(10))
        fig.show()

**Creamos los textboxes, botones y demas cosas necesarias para poder ingresar los datos de entrada y poder mostrar el sistema de recomendacion junto con la grafica**

In [8]:
#Textbox para ingresar el artista
artista_txtbox = widgets.Text(  
     placeholder = 'Taylor Swift',  
     description = 'Artista:',
     value = 'Taylor Swift',
)  
style = {'description_width': 'initial'}
#Slider para ingresar el minimo de generos que coincidan
genero_slider = widgets.IntSlider(description='Numero de generos', min = 1, max=5, style=style)
#Creamos el boton que presionaremos para obtener el sistema de recomendacion
boton_recomendar = Button(description='Recomendar Artistas')
#Esto hara que cada vez que presionemos de nuevo el boton el output antiguo sea borrado
outuser_info = widgets.Output()

#Indicamos que queremos cada vez que se de clic al boton, se llame a la funcion que realiza el sistema de recomendacion
boton_recomendar.on_click(sistema_recomendacion_artistas)

#Mostramos el textbox, el slider y el boton
display(artista_txtbox, genero_slider, boton_recomendar, outuser_info) 

Text(value='Taylor Swift', description='Artista:', placeholder='Taylor Swift')

IntSlider(value=1, description='Numero de generos', max=5, min=1, style=SliderStyle(description_width='initial…

Button(description='Recomendar Artistas', style=ButtonStyle())

Output()